## KNN Implementation

The implementation of k-Nearest Neighbors is divided into the following steps:

1. **Handle Data**: Open the dataset from CSV and split into test/train datasets.
2. **Similarity**: Calculate the distance between two data instances.
3. **Neighbors**: Locate k most similar data instances.
4. **Response**: Generate a response from a set of data instances.
5. **Accuracy**: Summarize the accuracy of the predictions.
6. **Main**: Tie it all together

### 1. Handle Data ###

The first thing we need to do is to load our data files. The files given to us are in `.arff` format. We can convert them into `.csv` format by changing the extension to `.csv` and removing the starting few lines upto `@data`. The data will now be in `csv` format without a header line or any quotes. We can now open the file using `open` function and read the data lines using the reader function in the `csv` module.

   We first need to convert the measures that were loaded as strings into numbers. Next we need to split the train dataset randomly into train and verification datasets. The ratio used is 80/20. At the end of this step we will have 3 arrays: trainSet, verSet and testSet.
    
   We can put this all together in two functions one for training data and the other for testing data and call them **loadTrainset** and **loadTestset**. **loadTrainset** loads a `CSV` with the provided file name and splits it randomly into train and verification datasets using the provided split ratio.
   **loadTestset** loads a `csv` with the provided file name and creates a test dataset.

In [42]:
import csv
import random
import math
import operator

def loadTrainset(filename, split, trainSet = [], verSet = []):
    with open(filename, 'rb') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        
        # Converting the strings into numbers
        for x in range(len(dataset)):
            for y in range(2, 6):
                dataset[x][y] = float(dataset[x][y])
            # splitting the data
            if random.random() < split:
                trainSet.append(dataset[x])
            else:
                verSet.append(dataset[x])
        

In [43]:
def loadTestset(filename, testSet = []):
    with open(filename, 'rb') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        
        # converting the strings into numbers
        for x in range(len(dataset)):
            for y in range(2, 6):
                dataset[x][y] = float(dataset[x][y])
            testSet.append(dataset[x])

### 2. Similarity ###

In order to make predictions we need to calculate the similarity between any two given data instances. This is so that we can locate the k most similar data instances in the training dataset for a given memeber of the test dataset and in turn make a prediction.

To measure the similarity between two instances we can use the Euclidean distance measure. Similarity score is the inverse of Euclidean distance. Larger Euclidean distance corresponds to smaller similarity score and vice-versa.

However on observing the data we notice that the first two features are not numbers and also that the remaining features vary over a large range. In order to account for these we do the following:

1. To find the euclidean distance we take the difference of the attribute values (between the test and train instances) in case of numeric attributes. For non-numeric attributes, we use the corresponding similarity value from the similarity matrix. According to the similarity matrix provided the value will be 1 when the attributes of train and test instances are equal. It will be 0 in all other cases. We use this value in the computation of euclidean distance
2. If features vary over a large range then the largest component will dominate the calculation of the similarity score. In order to avoid this we normalize the numerical attributes so that they fall between 0 and 1.

Putting all of this together we can define the **similarityScore** function as follows:

In [44]:
def similarityScore(instance1, instance2, trainSet, length):
    distance = 0
    for x in range(length):
        if (x >= 2 and x <= 5):
            xmax = max(l[x] for l in trainSet)
            xmin = min(l[x] for l in trainSet)
            distance += pow((instance1[x] - instance2[x])/(xmax - xmin), 2)
        else:
            if (instance1[x] != instance2[x]):
                distance += 1
    if(distance==0):
        return distance
    return 1/math.sqrt(distance)

### 3. Neighbors ###

We now use the similarity measure to collect the k most similar instances for a given unseen instance

This is done by calculating the similarity score corresponding to each class for all instances and selecting a subset with the largest similarity  score values.

**For example** 

(a) After sorting the scores, say the training vectors x1, x15, x20  were found to be the top 3-NN training data points close to the test vector (y)
Now say, you got the following similarity scores (inverse Euclidean distance given in the previous page) for the top 3 points: sim(x1,y)=3.2, sim(x15,y)=1.2, sim(x20,y)=3.7

(b) Say, in the training data, the class of x1 is C3 , class of x15 is C4, and the class of x20 is C3.  So, now you know that the test vector is either C3 or C4. You have to predict whether the class of  y  is C3 or C4 from just  x1, x15 and  x20

(c) prediction: 
argmax (   score for C3(=3.2+3.7) , score for C4 (=1.2)) 
= argmax(   score for C3(=6.9) ,   score for C4 (=1.2)           )
hence, class (or label) of y is C3, as 6.9>1.2 and the score is: 6.9 

Note:Meaning of argmax argmax stands for the argument of the maximum, i.e., the set of points for which the value of the given expression attains its maximum value

Below is the **getNeighbors** function that returns k most similar neighbors from the training set for a given test instance


In [52]:
def getNeighbors(trainSet, testInstance, k):
    distances = []
    length = len(testInstance) - 1
    for x in range(len(trainSet)):
        dist = similarityScore(testInstance, trainSet[x], trainSet, length)
        distances.append((trainSet[x], dist))
        
    distances.sort(key = operator.itemgetter(1), reverse=True)
    neighbors = []
    for x in range(k):
        category = distances[x][0][-1]
        sim_score = distances[x][1]
        neighbors.append((category,sim_score))
    
    return neighbors


### 4. Response ###

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.

We do this by allowing each neighbor to vote for their class attribute, and take the majority vote as the prediction.

In [53]:
# function for calculating the majority voted response from
# a number of neighbors.

# for each class, we add all the similarity scores
# The class with the maximum similarity score is our prediction
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        # get the class or category
        response = neighbors[x][0]
        if response in classVotes:
            classVotes[response] += neighbors[x][1]
        else:
            classVotes[response] = neighbors[x][1]
    sortedVotes = sorted(classVotes.iteritems(), key = operator.itemgetter(1), reverse = True)
    
    return sortedVotes[0][0]

### 5. Accuracy ###

To evaluate the accuracy of predictions we calculate the ratio of the total correct predictions out of all predictions made.

We implement the necessary logic in the **getAccuracy** function. The function sums up the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [54]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

### 6. Main ###

Now that we have all the elements of the algorithms we can tie them together with a main function

In [58]:
# main() function
def main():
    trainSet = []
    verSet = []
    split = 0.80
    loadTrainset('trainProdSelection.csv', split, trainSet, verSet)
    
    print 'Train: ' + repr(len(trainSet))
    print 'Verf: ' + repr(len(verSet))
    
    for x in range(len(trainSet) - 1):
        if (x >= 2 and x <= 5):
            xmax = max(l[x] for l in trainSet)
            xmin = min(l[x] for l in trainSet)
            print "Max: {}; Min: {}".format(xmax, xmin)
            
    # generate predictions
    predictions = []
    k = 5
    for x in range(len(verSet)):
        neighbors = getNeighbors(trainSet, verSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('> predicted=' + repr(result) + ', actual=' + repr(verSet[x][-1]))
    accuracy = getAccuracy(verSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')
        
main()

Train: 149
Verf: 37
Max: 64.0; Min: 2.0
Max: 347.0; Min: 3.0
Max: 31.55; Min: 8.8997
Max: 17.8737; Min: 0.008
> predicted='C1', actual='C1'
> predicted='C1', actual='C1'
> predicted='C1', actual='C1'
> predicted='C1', actual='C1'
> predicted='C1', actual='C1'
> predicted='C5', actual='C1'
> predicted='C2', actual='C2'
> predicted='C2', actual='C2'
> predicted='C2', actual='C2'
> predicted='C2', actual='C2'
> predicted='C2', actual='C2'
> predicted='C2', actual='C2'
> predicted='C2', actual='C2'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C3', actual='C3'
> predicted='C4', actual='C4'
> predicted='C1', actual='C4'
> predicted='C4', actual='C4'
> predicted='C4', actual='C4'
> predicted='C4', actual='C4'
> predicted='C1', ac